# TF 2.2 блокнот

In [ ]:
!pip install -q efficientnet

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.densenet import DenseNet201
from efficientnet.tfkeras import EfficientNetB7
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
#  библиотека для работы с наборами данных на Kaggle
from kaggle_datasets import KaggleDatasets
import re
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline 
print("Tensorflow version " + tf.__version__)

# Определяем, какой ускоритель можем использовать

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
# Обнаружение оборудования, возврат соответствующей стратегии распространения: TPU, GPU, CPU
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Обнаружение TPU. Параметры среды не требуются, если задана переменная среды TPU_NAME. На Kaggle это всегда так.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # стратегия распространения по умолчанию в Tensorflow. Работает на CPU и одном GPU.
print("REPLICAS: ", strategy.num_replicas_in_sync)

# Get my data path

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path("tpu-getting-started") #получаем путь к наборам данных

# Set some parameters

In [ ]:
IMAGE_SIZE = [512, 512] # при таком размере графическому процессору не хватит памяти. Используйте TPU
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

GCS_PATH_SELECT = { # available image sizes
    192: GCS_DS_PATH + '/tfrecords-jpeg-192x192',
    224: GCS_DS_PATH + '/tfrecords-jpeg-224x224',
    331: GCS_DS_PATH + '/tfrecords-jpeg-331x331',
    512: GCS_DS_PATH + '/tfrecords-jpeg-512x512'
}
GCS_PATH = GCS_PATH_SELECT[IMAGE_SIZE[0]]

VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test/*.tfrec')

SEED = 2020

# Загружаем данные

Эти данные загружаются из Kaggle и автоматически сегментируются для максимального распараллеливания.

In [ ]:
def decode_image(image_data):
    """Декодирует изображение в vyjujvthye. vfnhbwe (тензор)
    Нормализует данные и преобразовывает изображения к указанному размеру"""
    image = tf.image.decode_jpeg(image_data, channels=3) # Декодирование изображения в формате JPEG в тензор uint8.
    image = tf.cast(image, tf.float32) / 255.0  # преобразовать изображение в плавающее в диапазоне [0, 1]
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # явный размер, необходимый для TPU
#     image = tf.keras.applications.inception_resnet_v2.preprocess_input(image)
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string означает байтовую строку
        "class": tf.io.FixedLenFeature([], tf.int64),  # [] означает отдельный элемент
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT) # парсим отдельный пример в указанном формате
    image = decode_image(example['image']) # преобразуем изображение к нужному нам формату
    label = tf.cast(example['class'], tf.int32)
    return image, label # возвращает набор данных пар (изображение, метка)

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string означает байтовую строку
        "id": tf.io.FixedLenFeature([], tf.string),  # [] означает отдельный элемент
        # класс отсутствует, задача этого конкурса - предсказать классы цветов для тестового набора данных
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image']) # преобразуем изображение к нужному нам формату
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    """Читает из TFRecords. Для оптимальной производительности одновременное чтение из нескольких
    файлов без учета порядка данных. Порядок не имеет значения, поскольку мы все равно будем перетасовывать данные"""

    ignore_order = tf.data.Options() # Представляет параметры для tf.data.Dataset.
    if not ordered:
        ignore_order.experimental_deterministic = False # отключить порядок, увеличить скорость

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # автоматически чередует чтение из нескольких файлов
    dataset = dataset.with_options(ignore_order) # использует данные сразу после их поступления, а не в исходном порядке
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    # возвращает набор данных пар (изображение, метка), если метка = Истина, или пар (изображение, идентификатор), если метка = Ложь
    return dataset

def get_validation_dataset():
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=True)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache() # кешируем набор
    dataset = dataset.prefetch(AUTO) #готовим следующий набор, пока предыдущий обучается
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) #готовим следующий набор, пока предыдущий обучается
    return dataset
                               
def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

NUM_VALIDATION_IMAGES = count_data_items(VALIDATION_FILENAMES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
print('Dataset: {} validation images, {} unlabeled test images'.format(NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

# Построить модель на TPU (или GPU, или CPU...) с Tensorflow 2.1!

In [ ]:
def get_model(use_model):
    # noisy-student
    base_model = use_model(weights='imagenet', 
                      include_top=False, pooling='avg',
                      input_shape=(*IMAGE_SIZE, 3))
#     base_model.trainable = False
    x = base_model.output
    predictions = Dense(104, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(
                    optimizer='nadam',
                    loss = 'sparse_categorical_crossentropy',
                    metrics=['sparse_categorical_accuracy']
                 )
    return model
with strategy.scope():    
    model1 = get_model(DenseNet201) # тут подставить свою модель
model1.load_weights("/kaggle/input/more-data-with-densenet201/my_densenet_201.h5")

In [ ]:
def get_model(use_model):
    # noisy-student
    base_model = use_model(weights='noisy-student', 
                      include_top=False, pooling='avg',
                      input_shape=(*IMAGE_SIZE, 3))
#     base_model.trainable = False
    x = base_model.output
    predictions = Dense(104, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(
                optimizer='nadam',
                loss = 'sparse_categorical_crossentropy',
                metrics=['sparse_categorical_accuracy']
                )
    return model


with strategy.scope():    
    model2 = get_model(EfficientNetB7) # тут подставить свою модель
model2.load_weights("/kaggle/input/more-data-with-efficientnetb7/my_ef_net_b7.h5") 

In [ ]:
# from sklearn.metrics import f1_score
# val_dataset = get_validation_dataset()
# images_ds = val_dataset.map(lambda image, label: image)
# labels_ds = val_dataset.map(lambda image, label: label).unbatch()
# val_labels = next(iter(labels_ds.batch(NUM_VALIDATION_IMAGES))).numpy() # get everything as one batch
# m1 = model1.predict(images_ds)
# m2 = model2.predict(images_ds)
# scores = []
# for alpha in np.linspace(0,1,100):
#     val_probabilities = alpha*m1+(1-alpha)*m2
#     val_predictions = np.argmax(val_probabilities, axis=-1)
#     scores.append(f1_score(val_labels, val_predictions, labels=range(104), average='macro'))

# best_alpha = np.argmax(scores)/100
    
# print('Best alpha: ' + str(best_alpha))

# Вычислите свои прогнозы на тестовом наборе!

Cоздадим файл, который можно будет отправить на конкурс.

In [ ]:
# Поскольку мы разделяем набор данных и выполняем итерацию отдельно для изображений и идентификаторов, порядок имеет значение.
test_ds = get_test_dataset(ordered=True) 
best_alpha = 0.48
print('Вычисляем предсказания...')
test_images_ds = test_ds.map(lambda image, idnum: image)

probabilities1 = model1.predict(test_images_ds)
probabilities2 = model2.predict(test_images_ds)
probabilities = best_alpha * probabilities1 + (1 - best_alpha) * probabilities2
predictions = np.argmax(probabilities, axis=-1)
print(predictions)

print('Создание файла submission.csv...')
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # все в одной партии
np.savetxt('submission.csv', np.rec.fromarrays([test_ids, predictions]), fmt=['%s', '%d'], delimiter=',', header='id,label', comments='')